# Create Required Azure Services
Before starting this section, make sure you have set up the global variables from the previous section. Once your global variables have been set up, you no longer have to set them up again, as they are stored in a .env environment file. 

If you delete your .env file or your Azure login session expires, you may need to re-run the setup process as shown in the [previous section](02_setup_environment.ipynb). Keep this in mind when proceeding onto the other sections in this sample.

### Note:

If you already have resources setup that are ready to be used, please create a .env file in this folder and populate that file with the parameters along with their values as mentioned in the cells below along with the gloabl variable fro mthe previous section and you can skip running the cell in this notebook.

If you dont have the resources or an unsure, please follow the steps below to create them

## Get Global Variables
First, we will read the previously stored global variables.

In [ ]:
from env_variables import *

## Create Resource Group
A resource group is a container that holds related resources for an Azure solution. The resource group can include all the resources for the solution, or only those resources that you want to manage as a group. 
We will now create a new Azure resource group using the resource group name and location set in the [previous section](02_setup_environment.ipynb). Check that this name is present in the .env file, as this name will be auto-filled into the placeholders starting with the character "$". 

In [2]:
!az group create --name $resourceGroupName --location  $resourceLocation

{
  "id": "/subscriptions/e78c95ce-3ed7-49eb-a19e-8ebcc73bb608/resourceGroups/lvasample5d672f",
  "location": "southcentralus",
  "managedBy": null,
  "name": "lvasample5d672f",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


## Create Azure Container Registry Service
We will use Azure Container Registry (ACR) to store our module images.

In [3]:
!az acr create --name $acrServiceName --resource-group $resourceGroupName --location $resourceLocation --sku Basic --admin-enabled true

{- Finished ..
  "adminUserEnabled": true,
  "creationDate": "2020-07-06T21:09:39.808690+00:00",
  "id": "/subscriptions/e78c95ce-3ed7-49eb-a19e-8ebcc73bb608/resourceGroups/lvasample5d672f/providers/Microsoft.ContainerRegistry/registries/lvasample5d672fcontreg",
  "location": "southcentralus",
  "loginServer": "lvasample5d672fcontreg.azurecr.io",
  "name": "lvasample5d672fcontreg",
  "networkRuleSet": null,
  "policies": {
    "quarantinePolicy": {
      "status": "disabled"
    },
    "retentionPolicy": {
      "days": 7,
      "lastUpdatedTime": "2020-07-06T21:09:40.299195+00:00",
      "status": "disabled"
    },
    "trustPolicy": {
      "status": "disabled",
      "type": "Notary"
    }
  },
  "provisioningState": "Succeeded",
  "resourceGroup": "lvasample5d672f",
  "sku": {
    "name": "Basic",
    "tier": "Basic"
  },
  "status": null,
  "storageAccount": null,
  "tags": {},
  "type": "Microsoft.ContainerRegistry/registries"
}


> <span style="color:red; font-weight: bold"> [!WARNING] </span>  
> Even if the cell above finishes executing, it may take several seconds to minutes to have the changes reflected on Azure Datacenter. Wait for 2-3 minutes before proceeding to the next cell to avoid running into errors.

### Get Access Credentials from ACR
After the ACR resource has been generated, we can grab the access credentials from ACR by running the following code snippets.

In [4]:
%%bash --out output -s "$acrServiceName"
az acr credential show -n $1

In [5]:
import json
jsonRegDetails = json.loads(output)

acrUserName = jsonRegDetails['username']
acrPassword = jsonRegDetails['passwords'][0]['value']

Run the cell below to save these ACR credentials for future use by storing them in the .env file.

In [6]:
set_key(envPath, "CONTAINER_REGISTRY_USERNAME_myacr", acrUserName)
set_key(envPath, "CONTAINER_REGISTRY_PASSWORD_myacr", acrPassword)
tempVar = set_key(envPath, "ACR_SERVICE_FULL_NAME", acrServiceName+".azurecr.io")

## Create Azure IoT Hub
Next, we will create an Azure IoT Hub.
> <span style="color:red; font-weight: bold"> [!WARNING] </span>  
> Running the code snippet below may yield the following error: "IotHub name '<NAME\>' is not available". This error appears if the service already exists. If you see the error, skip the step, as you have likely created the resource already.


In [7]:
!az iot hub create --name $iotHubServiceName --resource-group $resourceGroupName --location $resourceLocation --sku S1

{- Finished ..
  "etag": "AAAAAD5cbEg=",
  "id": "/subscriptions/e78c95ce-3ed7-49eb-a19e-8ebcc73bb608/resourceGroups/lvasample5d672f/providers/Microsoft.Devices/IotHubs/lvasample5d672fiothub",
  "location": "southcentralus",
  "name": "lvasample5d672fiothub",
  "properties": {
    "authorizationPolicies": null,
    "cloudToDevice": {
      "defaultTtlAsIso8601": "1:00:00",
      "feedback": {
        "lockDurationAsIso8601": "0:00:05",
        "maxDeliveryCount": 10,
        "ttlAsIso8601": "1:00:00"
      },
      "maxDeliveryCount": 10
    },
    "comments": null,
    "deviceStreams": null,
    "enableFileUploadNotifications": false,
    "eventHubEndpoints": {
      "events": {
        "endpoint": "sb://iothub-ns-lvasample5-3777735-a4b0b36f84.servicebus.windows.net/",
        "partitionCount": 4,
        "partitionIds": [
          "0",
          "1",
          "2",
          "3"
        ],
        "path": "lvasample5d672fiothub",
        "retentionTimeInDays": 1
      }
    },
    "

Run the cells below to save these Azure IoT Hub connection string for future use by storing it in the .env file. This string is used to identify the hub.

In [8]:
%%bash --out iotHubConnString -s "$resourceGroupName" "$iotHubServiceName"

az iot hub show-connection-string --hub-name $2 --output tsv

In [9]:
tempVar = set_key(envPath, "IOT_HUB_CONN_STRING", iotHubConnString.rstrip())

## Create IoT Edge Device Identity
To run LVA on the Edge, we will need to create an IoT Edge device to run our modules with AI capability. In the code below, we will instruct Azure IoT Hub to create a resource framework for this cloud-based IoT Edge device.

In [10]:
# Install iot hub CLI extensions
!sudo az extension add --name azure-iot

Extension 'azure-iot' is already installed.


In [11]:
!az --version

azure-cli                         2.0.80 *

command-modules-nspkg              2.0.3
core                              2.0.80 *
nspkg                              3.0.4
telemetry                          1.0.4

Extensions:
azure-cli-ml                      1.0.85

Python location '/opt/az/bin/python3'
Extensions directory '/opt/az/extensions'

Python (Linux) 3.6.5 (default, Jan 13 2020, 05:04:39) 
[GCC 7.4.0]

Legal docs and information: aka.ms/AzureCliLegal



You have 2 updates available. Consider updating your CLI installation. Instructions can be found at https://docs.microsoft.com/en-us/cli/azure/install-azure-cli

Please let us know how we are doing: https://aka.ms/clihats


If we want to connect/match physical devices to this cloud-based Edge device later, the connection will be set using the IoT Edge device's connection string. Everything in this cloud-based Edge device will be cloned into the physical device, including all the IoT Edge modules. Thus, we will also be printing out and saving the cloud-based IoT Edge device's connection string for future use.

In [12]:
%%bash --out iotEdgeDeviceConnString -s "$resourceGroupName" "$iotHubServiceName" "$iotDeviceId"
# Create edge device if one does not already exist
sudo az iot hub device-identity show --resource-group $1 --hub-name $2 --device-id $3 &> /dev/null

if [ $? -ne 0 ]; then
    sudo az iot hub device-identity create --hub-name $2 --device-id $3 --edge-enabled &> /dev/null
fi

sudo az iot hub device-identity show-connection-string --hub-name $2 --device-id $3 --output tsv

Run the cell below to save the IoT Edge Device Connection String in the environment file for future use

In [13]:
tempVar = set_key(envPath, "IOT_EDGE_DEVICE_CONN_STRING", iotEdgeDeviceConnString.rstrip())

## Create Azure Storage Services
To store assets onto Azure Media Services, we must also create a storage resource.

In [14]:
!sudo az storage account create --name $storageServiceName --resource-group $resourceGroupName --location $resourceLocation --sku Standard_LRS

{- Finished ..
  "accessTier": null,
  "azureFilesIdentityBasedAuthentication": null,
  "creationTime": "2020-07-06T21:11:58.774880+00:00",
  "customDomain": null,
  "enableHttpsTrafficOnly": true,
  "encryption": {
    "keySource": "Microsoft.Storage",
    "keyVaultProperties": null,
    "services": {
      "blob": {
        "enabled": true,
        "keyType": "Account",
        "lastEnabledTime": "2020-07-06T21:11:58.868629+00:00"
      },
      "file": {
        "enabled": true,
        "keyType": "Account",
        "lastEnabledTime": "2020-07-06T21:11:58.868629+00:00"
      },
      "queue": null,
      "table": null
    }
  },
  "failoverInProgress": null,
  "geoReplicationStats": null,
  "id": "/subscriptions/e78c95ce-3ed7-49eb-a19e-8ebcc73bb608/resourceGroups/lvasample5d672f/providers/Microsoft.Storage/storageAccounts/lvasample5d672fstorage",
  "identity": null,
  "isHnsEnabled": null,
  "kind": "Storage",
  "largeFileSharesState": null,
  "lastGeoFailoverTime": null,
  "locatio

## Create Azure Media Services

Azure Media Services (AMS) is a cloud-based media workflow platform to index, package, protect, and stream video. For this sample, we will be using AMS to archive video clips (Edge streams) generated when our AI module detects motion.

In [15]:
!sudo az ams account create --name $mediaServiceName --resource-group $resourceGroupName --storage-account $storageServiceName --location $resourceLocation

Command group 'ams' is in preview. It may be changed/removed in a future release.
{
  "id": "/subscriptions/e78c95ce-3ed7-49eb-a19e-8ebcc73bb608/resourceGroups/lvasample5d672f/providers/Microsoft.Media/mediaservices/lvasample5d672fams",
  "location": "South Central US",
  "mediaServiceId": "1359a665-e9cd-4798-80ba-51812f4f525b",
  "name": "lvasample5d672fams",
  "resourceGroup": "lvasample5d672f",
  "storageAccounts": [
    {
      "id": "/subscriptions/e78c95ce-3ed7-49eb-a19e-8ebcc73bb608/resourceGroups/lvasample5d672f/providers/Microsoft.Storage/storageAccounts/lvasample5d672fstorage",
      "resourceGroup": "lvasample5d672f",
      "type": "Primary"
    }
  ],
  "tags": null,
  "type": "Microsoft.Media/mediaservices"
}


## Create Azure Service Principal
A service principal is a security identity used by user-created apps, services, and tools to access Azure resources. Like with the other resources created thus far, we will be creating a service principal and storing the credentials (AAD_TENANT_ID, AAD_SERVICE_PRINCIPAL_ID, and AAD_SERVICE_PRINCIPAL_SECRET) in the .env file. 


In [16]:
%%bash --out output -s "$mediaServiceName" "$resourceGroupName" 

sudo az ams account sp create --account-name $1 --resource-group $2 --name lvasp$2 --output json

if [ $? -ne 0 ]; then
    sudo az ams account sp reset-credentials --account-name $1 --resource-group $2 --name lvasp$2 --output json
fi

Command group 'ams' is in preview. It may be changed/removed in a future release.


In [17]:
import json

amssp = json.loads(output)

set_key(envPath, "AAD_TENANT_ID", amssp['AadTenantId'])
set_key(envPath, "AAD_SERVICE_PRINCIPAL_ID", amssp['AadClientId'])
tempVar = set_key(envPath, "AAD_SERVICE_PRINCIPAL_SECRET", amssp['AadSecret'])

## Next Steps
If all the code cells above have successfully finished running, return to the Readme page to continue.   